In [1]:
%run Latex_macros.ipynb

<IPython.core.display.Latex object>

# Bigger = Better ?  Scaling laws

There are many LLM's, with varying choices of
- number of parameters $N$
-size of training data $D$
    - number of tokens trained on
    - not *distinct* tokens in dataset
        - same token encountered in each epoch is counted once per epoch
- amount of compute for training $C$

Here is a table from the [GPT-3 paper](https://arxiv.org/pdf/2005.14165.pdf#page=46)

<img src="images/compute_by_model.png" width=90%>

We have already seen that some LLM properties
- like in-context learning (zero or few shot)
- "emerge" only when model size passes a threshold

This argues for bigger models.

<img src="images/LM_Few_Shot_Accuracy.png" width=80%>
                                           

There is also evidence that the emergence of ability to perform some in-context tasks
- is sudden
- rather than gradual
as the number of parameters increase.

<img src="images/arithmetic_LLM_by_size.png">

Attribution: [GPT-3 paper](https://arxiv.org/pdf/2005.14165.pdf#page=46)


Is bigger $N$ always better ?

Consider the costs.  Larger $N$
- entails more computation: larger $C$
- probably requires more training data: larger $D$

If we fix a "budget" for one choice (e.g., $C$) we can explore choices for $N, D$ that meet this budget.

Here are two models with the same $C$ budget
- but vastly different $N$ and $D$


model | Compute (PF-days) | params (M) | training tokens (B) |
:---|:---|:---|:---
RoBERTa-Large | 49.3 | 355 | 2000
GPT-3 2.7B    | 55.2 | 2650 | 300

Attribution: [GPT-3 paper](https://arxiv.org/pdf/2005.14165.pdf#page=46)

Given these choices: how do we choose ?

One way to quantify the decision is by setting a goal
- to maximize "performance"
- where this is usually proxied by "minimizing test loss" $L$
    - Cross Entropy for the "predict the next" token task of the LLM
 

We can state some basic theories
- Increasing $N$ creates the *potential* for better performance $L$
- To *actualize* the potential
    - we need increased $C$
        - more parameters via increasing the number of stacked Transformer Blocks
    - we need increased $D$

But this still leaves many unanswered questions
- Can $L$ always be reduced ?
    - Does performance hit a "ceiling" 
    - For a fixed $N$: perhaps increasing $D$ or $C$ won't help
- What is the relationship between $N$ and $D$ ?
    - how much must $D$ by increased when $N$ increases
- For a fixed $D$: what is the best choice for $N$ ?
    - holding performance constant

# Scaling Laws: early research

Fortunately, this
[paper](https://arxiv.org/pdf/2001.08361.pdf)
has 
- conducted an empirical study of models with varying $N, D, C$ and resulting $L$
- fit an empirical function (*Scaling Laws*) describing the dependency of $L$ on $N, D, C$.

We briefly summarize the results.

"Performance" (test loss $L$ ) depends on scale.

Scale consists of 3 components
- Compute $C$
- Dataset size $D$
- Parameters $N$

We can set a "budget" for any of variables $L, N, D, C$
- and examine trade-offs for the non-fixed variable


The paper shows that 
- Increasing your budget for one of the scale factors
- increases performance (decrease loss)
- **provided** the other two factors don't become bottlenecks

<img src="images/scaling_loss_v_scale.png">

But bottlenecks are a worry:
- The potential performance of a model of fixed size $N$ hits a "ceiling"
- That can't be overcome by increasing compute $C$

<img src="images/scaling_loss_vs_compute.png">

**Observation**

For a fixed Compute $C$
- a smaller model (that has reached its asymptotic minimum) has lower loss
- provided that there is enough training data

For a fixed $L$
- a smaller model reaches the loss with less compute

This is interesting in that more data $D$ may compensate for fewer parameters
- we may be able to create "small" models (fewer parameters)
- with performance equal to larger models
- given sufficient $D$


We can also set a performance budget $L$
- and examine the amount of training data $D$ to reach this budget
- as $N$ varies

<img src="images/scaling_loss_vs_datasize.png">

**Observation**

For a fixed $D$ 
- bigger models are more data efficient
    - for a given level of loss $L$, a larger model achieves $L$ with fewer tokens
- but at a higher $C$

Here is one graph that combines $N$ and $D$

<img src="images/scaling_loss_vs_D_and_N.png">

**Key result: how must $D$ scale with $N$ ?**

The authors show that Performance (test loss as a function of $N, D: L(N,D)$)
improves
- as long as $N$ and $D$ are scaled together
- optimal relationship

$$
\frac{N^{0.74}}{D} = \text{constant}
$$ 
   - Thus, if $N$ increases by a factor of $8$, $D$ should increase by a factor of $8^{0.74} \approx 5$
   
- Performance flattens if one of $N, D$ is fixed while the other increases



The [Scaling Laws](https://arxiv.org/pdf/2001.08361.pdf#page=4)
show that Loss follows a Power Law as a function of $N, C, D$.

[Here](https://arxiv.org/pdf/2001.08361.pdf#page=20) is a summary of the Scaling Laws.

<img src="images/scaling_power_laws_summary.png">

# Scaling laws: newer research

[Continuing research](https://arxiv.org/pdf/2203.15556.pdf) in the area of scaling
- confirms the need to scale $N$ and $D$ together
- but with a different scaling relationship

**Key result: how must $D$ scale with $N$ ?**

$$
\frac{N}{D} = \text{constant}
$$

Contrast this result with the original paper's relationship of the constant ratio as

$$
\frac{N^{0.74}}{D} = \text{constant}
$$



A key difference between the two papers is the *learning rate schedule*.

Recall: a learning rate moderates the rate a which gradient updates affect the model's weights during Gradient Descent

$$
\W_{(\tt+1)} = \W_\tp + \alpha_\tt * \frac{\partial \loss_\tp}{\partial \W}
$$

where $\alpha_\tp$ is the rate used at epoch $\tt$.

In the original paper, the learning rate schedule is *fixed* (constant across epochs)
$$
\alpha_\tp = c
$$

The newer paper shows that a fixed learning rate *over-estimates* $L(N,D)$ when $D \lt 130 B$
- leading to mis-fitting the empirical relationship

This can be avoided via a *variable learning rate* that decays $\alpha_\tp$ 
- to a fixed fraction of the initial rate $\alpha_{(0)}$
- as epoch number $\tt$ increases

Hence the optimal relationship changes from 
$\frac{N}{D} = \text{constant}$ to 
$\frac{N^{0.74}}{D} = \text{constant}$


As in the original paper, Test Loss is fit using empirical data as a function $L(N,D)$ of $N$ and $D$.
- but subject to a fixed compute budget $C$
- $L(N,D)$ is the early-stopped loss
    - not trained to optimal converged $L$
    - which would require more than the compute budget $C$
    
Given this function, one can find optimal $N$ and $D$ for a fixed compute budget $C$
 
$$
N_\text{opt}, D_\text{opt} = \argmin{ N, D \text{ s.t. } C=\text{FLOPS}(N,D)}{L(N,D)}
$$


This is a very interesting result.
- For someone on a fixed compute budget $C$
- One can find optimal values for model and data size

# The future of large models as seen through Scaling laws

The Scaling laws suggest
- given a fixed compute budget $C$
- there is an optimal number of parameters $N$ and number of training tokens $D$

Let's evaluate GPT-3, which pre-existed the scaling laws, in terms of what we now know.
- $N_\text{GPT} = 175B$
- $D_\text{GPT} = 0.3 T$

According to the Scaling Laws:
- GPT-3 is *under-trained* in time by a large factor
    $$\frac{C^*(N_\text{GPT},D_\text{GPT})}{C_\text{GPT}} = \frac{4.4 * 10^{24} \text{ Flops}}{3.1 * 10^{23}\text{Flops}} > 10$$
- GPT-3 is *under-trained* in number of tokens by a large factor

$$
\frac{D^*(N_\text{GPT})}{D_\text{GPT}}  = \frac{4.2 TB}{0.3 TB} \gt 10
$$

In order to take advantage of the 175 billion parameters, GPT-3 needed to be trained
- for much longer
- on many more tokens

Here is a plot of the optimal line
- we can see GPT-3 is above the line
- too large a $N$/too small a $D$
    - for the given $C$

<table>
    <tr>
        <center><strong>Chinchilla Optimal Training</strong></center>
    </tr>
    <tr>
        <img src="https://assets-global.website-files.com/621e749a546b7592125f38ed/62557f7626b9e103db549c7b_tokens_vs_flops%20(1).png" width=70%>
    </tr>
    <tr>
        Attribution: https://www.deepmind.com/blog/an-empirical-analysis-of-compute-optimal-large-language-model-training
    </tr>
</table>

One implication of these results is
- it may not be practical (in terms of compute budget) to *optimally* train models with $N \gt N_\text{GPT}$
- A 10 trillion parameter model needs 100 times the compute used for GPT-3

Given that reality, a likely future world is one of
- smaller $N$
- trained to optimality
- resulting in *better* performance $L$ than a larger model

The authors validated this hypothesis by comparing two models
- started with a large model called Gopher with $N_\text{Gopher} = 280B$
- trained a smaller model called Chinchilla with $N_\text{Chinchilla} = 70 B$
- using the same compute $C_\text{Chinchilla} = C_\text{Gopher}$
- but optimal $D$: $D_\text{Chinchilla} = 1.4 T$

Chinchilla, although only $25\%$ as large as Gopher
- outperforms on many benchmarks

The Scaling Laws are driving the design of new models.

- There are "clones" of GPT-3 with similar (or better) performance and many fewer parameters
    - at a greatly reduced compute budget.
    - [LLaMA](https://arxiv.org/pdf/2302.13971v1.pdf): 13B parameters
        - From Meta.  Model weights *are not* freely available
    - [BLOOM](https://huggingface.co/docs/transformers/model_doc/bloom)
        - family of models from the [BigScience Workshop](https://bigscience.huggingface.co/); Open-source 
- The successor ([PaLM 2](https://ai.google/static/documents/palm2techreport.pdf)) to Google's
$540B$ parameter PaLM model 
    - has only $16B$ parameters (in its largest configuration) but performs at a similar levelmm

Another trend (unrelated to scaling) is to incorporate *non-parametric* knowledge into models
- e.g., the Web as a source of "world" knowledge

With an external knowledge source, a model's parameters
- can be fewer
- encode "procedural" knowledge rather than factual knowledge
    
Thus, the trend towards models of ever increasing size is probably over.

# Inference budget

We have been focused on the *training budget*
- cost of a forward pass
- cost of a backward pass
- summed over many training examples


But larger models also require more compute (and greater latency) at *inference* time
- typical way of increasing model size is stacking more Transformer blocks
- deeper stack
    - greater latency
    - increased compute

Since we train a model once
- but perform inference many times
- perhaps we should focus on the *inference budget* rather than the *training budget*

A detailed examination of [inference cost](https://kipp.ly/transformer-inference-arithmetic/#flops-counting)
 approximates the number of Flops $F$ for once inference at
 $$
 F = n_\text{layers} * 24 * d_\text{model}^2
 $$
 
 Since $N$ is proportional to $n_\text{layers}$
 - inference cost is proportional to model size $N$
 
So a smaller model size $N$ can reduce our inference budget.

Recall
- a larger model achieves a given loss level *on fewer tokens* then a smaller model

But, fixing the loss level
- a smaller model can achieve the same loss
- at the cost of training on more tokens

The smaller model's inference cost is lower
- so trading off more tokens for fewer parameters
- may be *inference time* optimal
    

This idea was explored in ([LlaMa](https://arxiv.org/pdf/2302.13971.pdf))
- optimizes inference
- by training smaller models
- on *more data* than required for a larger model to achieve the same loss

<table>
    <tr>
        <center><strong>LlAMA loss vs training tokens</strong></center>
    </tr>
    <tr>
        <img src="images/llama-training-curves.png" width=50%>
    </tr>
    <tr>
        Attribution: https://arxiv.org/pdf/2302.13971.pdf#page=3
    </tr>
</table>

From the above graph: consider a Loss Level of 1.7
- the $N = $ 65B model requires $D = $ 600B tokens
- the $N = $ 33B model requires $D = $ 800B tokens

The author suggests that, for the same loss level, we can trade off
- doubling $N$
- for an increase of $D$ by $40 \%$

The result is that the current trend in LLM's is to
- small models
- trained on many tokens
- so as to optimize the inference budget


# Summary

The Scaling laws demonstrate that
- Larger models have the *potential* for smaller loss than smaller models
    - but require more  (compared to a smaller model) data and compute  to achieve their potential
- For a fixed Loss level, compared to a smaller model
    - larger models achieve the loss with fewer tokens (more data efficient)
    - **but** with a larger compute requirement (higher **training budget**)
    - a smaller model can achieve the same Loss
        - using more training tokens
        - **and** is more cost effective at inference time
        
Thus, the trend is toward smaller models.

In [2]:
print("Done")

Done
